In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
#df_org=pd.read_csv(filepath+"application_train_merge_clear_nomissing .csv")
df_org=pd.read_csv("application_train_merge_clear_nomissing_with_ylabel_dy.csv")

In [2]:
df=df_org

In [27]:
df.columns.tolist()

['Unnamed: 0',
 'TARGET',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_EDUCATION_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'HOUR_APPR_PROCESS_START_x',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'NAME_CONTRACT_TYPE_Cash loans',
 'NAME_CONTRACT_TYPE_Revolving loans',
 'CODE_GENDER_F',
 'CODE_GENDER_M',
 'FLAG_OWN_CAR_N',
 'FLAG_OWN_CAR_Y',
 'FLAG_OWN_REALTY_N',
 'FLAG_OWN_REALTY_Y',
 'NAME_TYPE_SUITE_Family',
 'NAME_TYPE_SUITE_Spouse, partner',
 'NAME_TYPE_SUITE_Unaccompanied',
 'NAME_INCOME_TYPE_Commercial associate',
 'NAME_INCOME_TYPE_Pensioner',
 'NAM

## 資料切割

In [3]:
y_=[]#找出 違約人的索引
for i in (df[df["simple_labelx"]!=0].index):
  y_.append(i)
y_1=[] #找出沒違約人的索引
for i in (df[df["simple_labelx"]==0].index):
  y_1.append(i)
df_split=df.loc[y_]
df_split0=df.loc[y_1]
df_tar1=df_split.reset_index()
df_tar0=df_split0.reset_index()
df_tar_e=df_tar0.iloc[42114:100000,:] #分成同等分
dfe=pd.concat([df_tar_e,df_tar1],ignore_index=True,axis=0)

In [63]:
100000-57886

42114

In [4]:
## RFE randomforest選出來的前十個(從全部特徵裡選出來的)with prev
df_X = dfe[['DAYS_LAST_PHONE_CHANGE', 'PREV_ANNUITY_median', 'PREV_APPLICATION_max',
       'PREV_CREDIT_max', 'PREV_CREDIT_sum', 'DAYS_DECISION_median',
       'CONTRACT_Cash_loans', 'CONTRACT_Consumer_loans', 'PORTFOLIO_Cash',
       'YIELD_high']]
df_y = dfe['simple_labelx']

In [13]:
df_X=dfe[['DAYS_LAST_PHONE_CHANGE',
       'AMT_REQ_CREDIT_BUREAU_YEAR']]
df_y = dfe['simple_labelx']

In [ ]:
df_X = dfe[['AMT_INCOME_TOTAL',  'DAYS_REGISTRATION',
        'CNT_FAM_MEMBERS',
       'OBS_60_CNT_SOCIAL_CIRCLE']]
df_y = dfe['simple_labelx']

In [17]:
df.head()

,Unnamed: 0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,simple_labelx,scores
0,0,1.0,0.0,202500.0,1.0,-9461.0,-637.0,-3648.0,-2120.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,96
1,1,0.0,0.0,270000.0,3.0,-16765.0,-1188.0,-1186.0,-291.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,100
2,2,0.0,0.0,67500.0,1.0,-19046.0,-225.0,-4260.0,-2531.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100
3,3,0.0,0.0,135000.0,1.0,-19005.0,-3039.0,-9833.0,-2437.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96
4,4,0.0,0.0,121500.0,1.0,-19932.0,-3038.0,-4311.0,-3458.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88


## Try smote

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X , y = dfe.iloc[:,2:-2] , dfe.iloc[:,-2]
#X , y = df_X  , df_y
stdsc = StandardScaler()
X = stdsc.fit_transform(X)
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.1,#超過10萬9,1分比較好 
                     #random_state=0, 
                     stratify=y)




In [29]:
from imblearn.over_sampling import SMOTE
X_res,y_res = SMOTE(random_state=41).fit_resample(X_train,y_train)

(449324, 80)

(449324,)

0.0    224662
1.0     52097
Name: simple_labelx, dtype: int64

(449324, 80)

In [38]:
from xgboost import XGBClassifier

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
# 使用訓練資料訓練模型
xgboostModel.fit(X_res, y_res)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(X_res)

In [47]:
# 預測成功的比例
print('訓練集: ',xgboostModel.score(X,y))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  0.812770925267714
測試集:  0.8116220083246618


In [50]:
print('分類的預測結果：')
pred = xgboostModel.predict(X) 
print(pred) #觀察預測結果AA
print('真實數據：')
print(y.values)  #觀察真實數據(Test data)

分類的預測結果：
[0 0 0 ... 0 0 0]
真實數據：
[0. 0. 0. ... 1. 0. 1.]


In [51]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, pred)

array([[249556,     69],
       [ 57506,    380]])

0.19168666666666667

In [66]:
from xgboost import XGBClassifier

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
# 使用訓練資料訓練模型
xgboostModel.fit(X_train, y_train)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(X_train)

In [67]:
# 預測成功的比例
print('訓練集: ',xgboostModel.score(X_train,y_train))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  0.6777933470257405
測試集:  0.6014855760925893


In [57]:
xgboostModel.feature_importances_

array([0.04752251, 0.01133161, 0.01091016, 0.01553189, 0.01433255,
       0.01110491, 0.01134785, 0.011256  , 0.00801148, 0.01447892,
       0.01367397, 0.01770196, 0.00998281, 0.01018202, 0.01047947,
       0.01010938, 0.00725966, 0.01139669, 0.0123744 , 0.0099507 ,
       0.01653048, 0.04813385, 0.01028437, 0.01113081, 0.01258566,
       0.05456142, 0.05918996, 0.        , 0.00785614, 0.00250714,
       0.00944629, 0.        , 0.01555485, 0.        , 0.01065454,
       0.01033541, 0.01118081, 0.0093401 , 0.        , 0.00930007,
       0.00936617, 0.01149656, 0.0111186 , 0.00878105, 0.00962319,
       0.00994606, 0.01280336, 0.01497433, 0.01432325, 0.01384071,
       0.01240824, 0.01025764, 0.01356657, 0.00856376, 0.01082209,
       0.00980794, 0.01119874, 0.01248373, 0.00915094, 0.00914285,
       0.00888859, 0.01040812, 0.00965207, 0.01014216, 0.00967482,
       0.00946878, 0.01329012, 0.01074098, 0.0123577 , 0.01089275,
       0.00811211, 0.01069538, 0.        , 0.00999248, 0.01100

## Stacking

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor


estimators = [
    ('rf', RandomForestRegressor(random_state = 42)),
    ('svr', svm.SVR())
    
]
clf = StackingRegressor(
    estimators=estimators, final_estimator=DecisionTreeRegressor(random_state = 42),
                            )


clf.fit(X_train, y_train)

print("訓練集 Score: ", clf.score(X_train,y_train))
print("測試集 Score: ", clf.score(X_test,y_test))